In [2]:
import torch
import torch.nn as nn
import DiscreteVariationalParameterizations as DVP
from SSEPDatasetGenerator import SEPGenerator
from GibbsSampling import BatchedConditionalGibbsSampler
from TrainDiscreteEmbeddingMI import EmbeddingMI1

model = EmbeddingMI1(256, 32, 32)    
model.load_state_dict(torch.load('/home/azureuser/cloudfiles/code/32_32.model'))
model = model.cuda()

/mnt/batch/tasks/shared/LS_root/mounts/clusters/nhazra1/code/Users/nhazra/SeniorThesis/DiscreteVariationalParameterizations.py:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.b)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/nhazra1/code/Users/nhazra/SeniorThesis/DiscreteVariationalParameterizations.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.c)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/nhazra1/code/Users/nhazra/SeniorThesis/DiscreteVariationalParameterizations.py:21: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.W, gain=1)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/nhazra1/code/Users/nhazra/SeniorThesis/DiscreteVariationalParameterizations.py:98: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavi

In [3]:
gen = SEPGenerator(space_size=32, num_samples=256, time_period=0,
                           inverse_density=4)

initial_state = gen.data.clone().cuda().float()
gen.run()
final_state = gen.data.clone().cuda().float()

print(initial_state[0])
print(final_state[0])

tensor([1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
       device='cuda:0')
tensor([1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
       device='cuda:0')


In [4]:
W = model.encoder_decoder.encoder_sample_(initial_state)
Z = model.encoder_decoder.encoder_sample_(final_state)
print(W[0])
print(Z[0])

tensor([0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.],
       device='cuda:0')
tensor([0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.],
       device='cuda:0')


In [10]:
x_w = model.encoder_decoder.conditional_log_probability_x_given_w_(W, initial_state)
w_x = model.encoder_decoder.conditional_log_probability_w_given_x_(W, initial_state)
# initial state needed for proposal distribution
w_z = model.embedding_dynamics.estimated_normalized_log_probabilities_w_given_z_better_(Z, W, initial_state, model)
loss = x_w - w_x + w_z
print('Mutual Information', 16.16 + loss.mean().detach().cpu().item())
print('Information Lost in Nats', -loss.mean().detach().cpu().item())

Mutual Information 15.493816349506378
Information Lost in Nats 0.6661836504936218


In [6]:
print(model.encoder_decoder.W)

Parameter containing:
tensor([[[ -0.7162,  -0.6939,  -0.5457,  ...,  -8.0163,  -0.9889,  -0.7271],
         [  0.8929,   1.4063,   1.2980,  ...,   1.7970,   0.9372,   1.7802],
         [ -1.0806,  -1.9294,  -1.7387,  ...,  -1.6820,  -1.0386,  24.6036],
         ...,
         [ -1.2108,  -1.4533,  -1.0337,  ...,  -1.8103,  -1.2306,  -1.3106],
         [  1.0834,   0.5674,   0.7735,  ...,   1.2115, -13.6508,   0.8256],
         [  0.4552,   0.4463,   0.6506,  ...,   0.5289,   0.8511,   0.9722]]],
       device='cuda:0', requires_grad=True)
